In [ ]:
# 必要なライブラリのインストール
!pip install lightgbm
!pip install signate

In [ ]:
# signate api を一回実行しておく
!signate list

In [ ]:
# token
!echo '作成したトークンをここに記載する' > ~/.signate/signate.json

In [ ]:
# コンペデータのダウンロード
!signate download -c 294

In [ ]:
# ライブラリのインポート
import numpy as np
import pandas as pd

import sklearn
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb

In [ ]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [ ]:
train.info()

In [ ]:
# 訓練データ作成
train['employment_length'] = train['employment_length'].fillna('non')
train.loc[:,"loan_status"] = train.loc[:,"loan_status"].replace({"FullyPaid": 0, "ChargedOff": 1})
train = pd.get_dummies(train, columns=["term", "grade", "employment_length", "purpose", "application_type"])

test['employment_length'] = test['employment_length'].fillna('non')
test = pd.get_dummies(test, columns=["term", "grade", "employment_length", "purpose", "application_type"])

Y = train.loc[:,"loan_status"]
X = train.drop(columns=["id", "loan_status"])
Z = test.drop(columns=["id"])

In [ ]:
# 訓練
gbm = lgb.train(
    params={'metric': 'binary'},
    train_set=lgb.Dataset(X, label=Y)
)
pred = gbm.predict(Z)

In [ ]:
# 提出データ作成
answer = pd.read_csv('./submit.csv', header=None)
answer[1] = np.where(pred < 0.2186295121329528, 0, 1) # 閾値はオプティマイザで作成

answer.to_csv('./answer.csv',  index=False, header=None)

In [ ]:
answer

In [ ]:
# 提出
!signate submit -c 294 './answer.csv'

In [ ]:
# 特徴量選択、特徴量の重要度
importance = pd.DataFrame(gbm.feature_importance(importance_type='gain'), index=X.columns, columns=['importance'])

In [ ]:
importance.plot.barh()